In [2]:
# !pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [9]:
import sys
import torch
import torch.optim as optim
from detectron2.utils.events import (
    CommonMetricPrinter,
    EventStorage
)

from detectron2 import model_zoo
import detectron2.utils.comm as comm
from detectron2.modeling import build_model
from detectron2.data import transforms as T_
from detectron2.config import get_cfg
from detectron2.data.datasets.coco import register_coco_instances
from detectron2.utils.logger import setup_logger
from detectron2.data import DatasetMapper
from detectron2.data import detection_utils as utils
from detectron2.data import transforms as T_
from detectron2.data import (
    DatasetCatalog,
    MetadataCatalog,
    MapDataset,
    get_detection_dataset_dicts )
from detectron2.checkpoint import DetectionCheckpointer
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
main_dir = '/content/drive/MyDrive/'
img_dir = 'data/face_mask_detection/images/'
ref_dir = 'Reference/'
panoptic_dir = 'Dataset/Panoptic'
semantic_dir = 'Dataset/Semantic'
model_dir = 'models/'

meta = {
    "thing_classes": ["with_mask", "without_mask", "mask_weared_incorrect"],
    "thing_colors": [[60, 179, 113], [75, 190, 123], [0, 240, 110]],
    "stuff_classes": ["Things", "Background"],
    "stuff_colors": [[82, 18, 128], [135, 206, 250]],
    "thing_dataset_id_to_contiguous_id": {1: 0, 2:1, 3:2},
    "stuff_dataset_id_to_contiguous_id": {0: 0, 1: 1},
}

In [3]:
DatasetCatalog.clear()
MetadataCatalog.clear()

image_root = main_dir + img_dir
register_coco_instances(name="mask_train",
                          metadata=meta,
                          image_root=image_root,
                          json_file=main_dir + "coco_train_set.json")

register_coco_instances(name="mask_val",
                          metadata=meta,
                          image_root=image_root,
                          json_file=main_dir + "coco_val_set.json")


register_coco_instances(name="mask_test",
                          metadata=meta,
                          image_root=image_root,
                          json_file=main_dir + "coco_test_set.json")

In [10]:
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file('COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml'))
cfg.MODEL.WEIGHTS = main_dir + model_dir + "model_final_f10217.pkl"

# Default is BGR as input although they internally convert it to RGB
cfg.INPUT.FORMAT = "RGB"

cfg.MODEL.DEVICE = "cuda"

# cfg.MODEL.PIXEL_MEAN = pix_mean
# cfg.MODEL.PIXEL_STD = pix_std

# #To use as Cascade
# cfg.MODEL.ROI_HEADS.NAME = "CascadeROIHeads"
# cfg.MODEL.ROI_BOX_HEAD.CLS_AGNOSTIC_BBOX_REG = True


cfg.MODEL.BACKBONE.FREEZE_AT = 0
cfg.MODEL.RESNETS.NORM = "GN"
cfg.MODEL.FPN.NORM = "GN"
cfg.MODEL.ROI_BOX_HEAD.NORM = "GN"
cfg.MODEL.ROI_MASK_HEAD.NORM = "GN"
cfg.MODEL.SEM_SEG_HEAD.NORM = "GN"


cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
# cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES = 2

cfg.SOLVER.IMS_PER_BATCH = 2

# cfg.MODEL.ROI_BOX_HEAD.BBOX_REG_LOSS_TYPE = roi_box_head_loss
# cfg.MODEL.ROI_BOX_HEAD.BBOX_REG_LOSS_WEIGHT = float(roi_box_head_loss_weight)
# cfg.MODEL.RPN.BBOX_REG_LOSS_TYPE = rpn_loss
# cfg.MODEL.RPN.BBOX_REG_LOSS_WEIGHT = float(rpn_loss_weight)




model = build_model(cfg)

DetectionCheckpointer(model).load(cfg.MODEL.WEIGHTS)

model.to(device)

AssertionError: ignored